In [13]:
import os
import io
import pandas as pd
from pandas import DataFrame
import sys
import numpy as np
import json
import pickle
import jsonpickle
import time
import random

import google.cloud
from google.cloud import datastore
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

from sqlalchemy import *
import sqlalchemy
import sqlalchemy.schema
from sqlalchemy.types import *

from IPython.display import *
from pandas.io.json import *
# gcloud auth application-default login

In [14]:
#df = pd.read_pickle('/home/jm/Documents/testingStuff/my_file.pkl')
engine = create_engine('mysql+pymysql://root:@localhost/sawi_tweets?charset=utf8mb4', encoding='utf8', echo = False)
df = pd.read_sql_table("sawi_tweets_historical", con = engine)

/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_1` CHECK (`user_verified` in (0,1)),'
  util.warn("Unknown schema content: %r" % line)
/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_2` CHECK (`RT` in (0,1))'
  util.warn("Unknown schema content: %r" % line)


In [15]:
df.head()

,index,created_at,favorite_count,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_count,screen_name,text,tweet_id,...,user_favorites_count,user_followers_count,user_friends_count,user_id,user_location,user_name,user_statuses_count,user_time_zone,user_verified,RT
0,0,2017-09-24 23:54:27,2,None,None,None,0,AmanteTech,Would next #Apple #iOS update cause your exist...,912102964881367040,...,2133,635,632,869739471570325504,"Sydney, New South Wales",AmanteTech,6314,Pacific Time (US & Canada),0,0
1,1,2017-09-24 23:30:33,0,None,None,None,84,supermeg500,RT @MegsSkunkey: iPhone 8 giveaway!!\nhttps://...,912096951532847104,...,61,361,4512,819370348420284417,,Giveaways~,402,None,0,1
2,2,2017-09-24 22:45:06,0,None,None,None,1860,botlamaya,RT @Sudathchamin: iPhone X new price?Ur left k...,912085514282553344,...,10,617,274,941255144,Colombo,Bot Lamaya [beta],51626,New Delhi,0,1
3,3,2017-09-24 22:45:06,0,None,None,None,1860,hikma_5000,RT @Sudathchamin: iPhone X new price?Ur left k...,912085512776798213,...,1388,143,113,791115735200182272,"Mansfield, TX",Hikma🌹🌹🌹,5432,Pacific Time (US & Canada),0,1
4,4,2017-09-24 22:36:56,0,None,None,None,4,Joey2031,RT @BoinkHunter: Beautiful device. \n#iPhoneX ...,912083456552783873,...,853,66,75,705859071044423680,,Joey,646,None,0,1


In [16]:
df.describe()

,index,favorite_count,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count,user_verified,RT
count,1184.000000,1184.000000,1184.000000,1.184000e+03,1184.000000,1184.000000,1184.000000,1.184000e+03,1.184000e+03,1184.000000,1184.000000
mean,45.921453,0.357264,54.638514,9.119618e+17,3992.051520,3291.594595,1595.056588,3.645501e+17,2.664078e+04,0.010135,0.545608
std,28.067900,1.679240,227.904993,9.066838e+14,20580.171232,24468.556120,4993.703040,4.247335e+17,1.058650e+05,0.100204,0.498126
min,0.000000,0.000000,0.000000,9.102936e+17,0.000000,0.000000,0.000000,6.728130e+05,1.000000e+00,0.000000,0.000000
25%,22.000000,0.000000,0.000000,9.111485e+17,30.000000,19.000000,57.000000,2.216952e+08,3.675000e+02,0.000000,0.000000
50%,45.000000,0.000000,2.000000,9.120673e+17,250.000000,210.000000,241.000000,3.246030e+09,2.359500e+03,0.000000,1.000000
75%,69.000000,0.000000,84.000000,9.127677e+17,1568.250000,1380.250000,1092.500000,8.698359e+17,7.930000e+03,0.000000,1.000000
max,99.000000,25.000000,2427.000000,9.135381e+17,535299.000000,781699.000000,52357.000000,9.127624e+17,1.693118e+06,1.000000,1.000000


In [17]:
df.count()

index                      1184
created_at                 1184
favorite_count             1184
in_reply_to_screen_name       4
in_reply_to_status_id         4
in_reply_to_user_id           4
retweet_count              1184
screen_name                1184
text                       1184
tweet_id                   1184
tweet_location                5
user_created_at            1184
user_description           1184
user_favorites_count       1184
user_followers_count       1184
user_friends_count         1184
user_id                    1184
user_location              1184
user_name                  1184
user_statuses_count        1184
user_time_zone              619
user_verified              1184
RT                         1184
dtype: int64

In [18]:
df.drop_duplicates(inplace=True)